# CPA

Jos jedan od nacina za napadanje implementacija kriptografskih algoritama je Correlation Power Attack. Posto je za promenu stanja na data busevima potrebna energija, mozemo videti da postoji veza izmedju broja jedinica na magistrali i potrosnje energije- sto vise jedinica, veci je siljak na grafu.
Na osnovu ove cinjenice, moguce je napraviti model potrosnje energije. 

Koraci CPA su sledeci:
* Slanje nasumicnih podataka cipu na enkripciju
* Merenje potrosnje energije u toku enkripcije
* Pogadjanje bajta kljuca (256 mogucnosti) i procena potrosnje energije na osnovu modela potrosnje (Hamingove tezine)
* Racunanje Pirsonovog koeficijenta korelacije izmedju modelovane i stvarne potrosnje energije za svaki bajt
* Odabir bajta koji ima najvecu korelaciju sa stvarnim (izmerenim) vrednostima
* Ponavljanje koraka 3, 4 i 5 za ostale bajtove kljuca

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
import numpy as np

In [3]:
def load_training_shards(training_data_path, num_shards=256):
  shard_array = []
  for shard_idx, shard_name in enumerate(os.listdir(training_data_path)):
    if shard_idx == num_shards:
      break
    shard_array.append(np.load(training_data_path + '/' + shard_name))

  return shard_array

In [4]:
training_shards = load_training_shards('/content/drive/MyDrive/datasets/tinyaes/train', num_shards=50)

In [5]:
import random

test_shard = training_shards[random.randint(0, len(training_shards))]

In [6]:
correct_key = test_shard['keys'][:,0]
print(f"Correct key: {correct_key}")

Correct key: [158 195 131  15 197 104 201  18 150  64  82  34 109 212 235 205]


In [7]:
AES_SBOX = [
    # 0    1    2    3    4    5    6    7    8    9    a    b    c    d    e    f 
    0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76, # 0
    0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0, # 1
    0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15, # 2
    0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75, # 3
    0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84, # 4
    0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf, # 5
    0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8, # 6
    0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2, # 7
    0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73, # 8
    0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb, # 9
    0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79, # a
    0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08, # b
    0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a, # c
    0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e, # d
    0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf, # e
    0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16  # f
]

def process_aes(input_byte, key_byte):
  sbox_input = input_byte ^ key_byte
  return AES_SBOX[sbox_input]

Iz same implementacije elektronskih komponenata sistema proizilazi cinjenica da se sa vecim brojem jedinica u binarnom zapisu podataka trosi vise struje. CPA koristi ovu cinjenicu da modeluje potrosnju. Broj jedinica nekog broja se naziva Hamingova tezina.

In [8]:
def hamming_weight(num):
  hw = bin(num).count('1')
  return hw

In [9]:
hamming_weight(0x24)

2

In [10]:
test_shard['pts'][1].shape

(256,)

In [11]:
from tqdm import trange

# Attack single byte
key_byte_index = 0

traces = test_shard['traces'][:,:,0]
traces_mean = np.mean(test_shard['traces'], axis=0)
traces_stddev = np.std(test_shard['traces'], axis=0)

traces_diff = traces - traces_mean.transpose()

cpa_outputs = 256*[0]

for key_guess in trange(0,256, position=0, leave=True):

  hamming_weights = [hamming_weight(process_aes(byte, key_guess)) for byte in test_shard['pts'][key_byte_index]]

  # Pearson's correlation coefficient:
  # 
  #
  # r = cov(X,Y) / stddev(X)*stddev(Y)
  # 
  # cov(X,Y) = E[(X-X_)*(Y-Y_)]   => sum((X-))
  # stddev(X) = sqrt((X-X_)^2)

  hamming_weights_mean = np.mean(hamming_weights)
  hamming_weights_stddev = np.std(hamming_weights)
  hamming_weights_diff = (hamming_weights - hamming_weights_mean).reshape((256,1))
  
  covariance = np.sum(hamming_weights_diff*traces_diff, axis=0)
  pearson_correlation = covariance/((hamming_weights_stddev * traces_stddev).transpose())
  
  # Point in the trace where the correlation is biggest -> where the leak is happening
  cpa_outputs[key_guess] = np.max(abs(pearson_correlation))

predicted_key_values = np.asarray(cpa_outputs)[np.argsort(cpa_outputs)][::-1]
print(f"Top 5 key byte predictions: {predicted_key_values[:5]}")

print(f"Correct key byte: {correct_key[key_byte_index]}")

100%|██████████| 256/256 [00:28<00:00,  9.08it/s]

Top 5 key byte predictions: [223.83185125  75.16675114  74.4665127   74.02666667  73.07912585]
Correct key byte: 158


In [12]:
cpa_results_indices = np.argsort(cpa_outputs)[::-1]
cpa_results_coefficients = np.asarray(cpa_outputs)[cpa_results_indices]

print(f"Top 5 key byte predictions: {cpa_results_indices[:5]} with respective correlation values: {cpa_results_coefficients[:5]}")

print(f"Correct key byte: {correct_key[key_byte_index]}")

Top 5 key byte predictions: [158 254 194  46 131] with respective correlation values: [223.83185125  75.16675114  74.4665127   74.02666667  73.07912585]
Correct key byte: 158


In [ ]:
# Full attack:
from tqdm.notebook import tqdm

key_predictions = np.zeros((16, 256))

for key_byte_index in tqdm(range(16), position=0, desc=f"Attacking key byte {key_byte_index}", leave=False, colour='green', ncols=80):

  for key_guess in tqdm(range(256), position=1, desc=f"Key guess: {hex(key_guess)}", leave=False, colour='red', ncols=80):

    hamming_weights = [hamming_weight(process_aes(byte, key_guess)) for byte in test_shard['pts'][key_byte_index]]

    # Pearson's correlation coefficient:
    # 
    #
    # r = cov(X,Y) / stddev(X)*stddev(Y)
    # 
    # cov(X,Y) = E[(X-X_)*(Y-Y_)]   => sum((X-))
    # stddev(X) = sqrt((X-X_)^2)

    hamming_weights_mean = np.mean(hamming_weights)
    hamming_weights_stddev = np.std(hamming_weights)
    hamming_weights_diff = (hamming_weights - hamming_weights_mean).reshape((256,1))
    
    covariance = np.sum(hamming_weights_diff*traces_diff, axis=0)
    pearson_correlation = covariance/((hamming_weights_stddev * traces_stddev).transpose())
    
    # Point in the trace where the correlation is biggest -> where the leak is happening
    key_predictions[key_byte_index][key_guess] = np.max(abs(pearson_correlation))

Attacking key byte 15:   0%|                             | 0/16 [00:00<?, ?it/s]

Key guess: 0xff:   0%|                                  | 0/256 [00:00<?, ?it/s]

Key guess: 0xff:   0%|                                  | 0/256 [00:00<?, ?it/s]

Key guess: 0xff:   0%|                                  | 0/256 [00:00<?, ?it/s]

Key guess: 0xff:   0%|                                  | 0/256 [00:00<?, ?it/s]

...

Key guess: 0xff:   0%|                                  | 0/256 [00:00<?, ?it/s]

Key guess: 0xff:   0%|                                  | 0/256 [00:00<?, ?it/s]

Key guess: 0xff:   0%|                                  | 0/256 [00:00<?, ?it/s]

Key guess: 0xff:   0%|                                  | 0/256 [00:00<?, ?it/s]

Key guess: 0xff:   0%|                                  | 0/256 [00:00<?, ?it/s]

In [34]:
def verify_CPA(correct_key, key_predictions):
  for i, k in enumerate(key_predictions):
    cpa_results_indices = np.argsort(k)[::-1]
    cpa_results_coefficients = np.asarray(k)[cpa_results_indices]

    if correct_key[i] == cpa_results_indices[0]:
      print(f"✔️ Key byte {cpa_results_indices[0]} guessed - pearson's coefficient = {cpa_results_coefficients[0]}")
    else:
      print(f"❌ Predicted byte {cpa_results_indices[0]}, but correct byte is {correct_key[i]}")

In [35]:
verify_CPA(correct_key, key_predictions)

✔️ Key byte 158 guessed - pearson's coefficient = 223.83185125303152
✔️ Key byte 195 guessed - pearson's coefficient = 221.09431021044426
✔️ Key byte 131 guessed - pearson's coefficient = 227.65344129554657
✔️ Key byte 15 guessed - pearson's coefficient = 220.06124093473005
✔️ Key byte 197 guessed - pearson's coefficient = 172.67498643516006
✔️ Key byte 104 guessed - pearson's coefficient = 186.10026954177897
✔️ Key byte 201 guessed - pearson's coefficient = 203.5997544505832
✔️ Key byte 18 guessed - pearson's coefficient = 191.83534421575587
✔️ Key byte 150 guessed - pearson's coefficient = 168.0567711413365
✔️ Key byte 64 guessed - pearson's coefficient = 197.16561656165618
✔️ Key byte 82 guessed - pearson's coefficient = 234.61333333333334
✔️ Key byte 34 guessed - pearson's coefficient = 188.8240534521158
✔️ Key byte 109 guessed - pearson's coefficient = 190.21648745519713
✔️ Key byte 212 guessed - pearson's coefficient = 190.82033898305085
✔️ Key byte 235 guessed - pearson's coeffi

In [ ]:
def CPA_attack(traces, plaintext, aes_key_size=16):
  
  num_traces = traces.shape[0]
  key_predictions = np.zeros((aes_key_size, num_traces))
  
  traces_mean = np.mean(traces, axis=0)
  traces_stddev = np.std(traces, axis=0)

  traces_diff = traces - traces_mean.transpose()

  for key_byte_index in tqdm(range(16), position=0, desc=f"Attacking key byte", leave=False, colour='green', ncols=80):

    for key_guess in tqdm(range(256), position=1, desc=f"Key guess", leave=False, colour='red', ncols=80):

      hamming_weights = [hamming_weight(process_aes(byte, key_guess)) for byte in plaintext[key_byte_index]]

      # Pearson's correlation coefficient:
      # 
      #
      # r = cov(X,Y) / stddev(X)*stddev(Y)
      # 
      # cov(X,Y) = E[(X-X_)*(Y-Y_)]   => sum((X-))
      # stddev(X) = sqrt((X-X_)^2)

      hamming_weights_mean = np.mean(hamming_weights)
      hamming_weights_stddev = np.std(hamming_weights)
      hamming_weights_diff = (hamming_weights - hamming_weights_mean).reshape((num_traces,1))
      
      covariance = np.sum(hamming_weights_diff*traces_diff, axis=0)
      pearson_correlation = covariance/((hamming_weights_stddev * traces_stddev).transpose())
      
      # Point in the trace where the correlation is biggest -> where the leak is happening
      key_predictions[key_byte_index][key_guess] = np.max(abs(pearson_correlation))
  
  return key_predictions

In [ ]:
# Lower the number of traces and try again
cpa_input_traces = test_shard['traces'][:256,:,0]
cpa_input_plaintexts = test_shard['pts'][:,:256]

cpa_result = CPA_attack(cpa_input_traces, cpa_input_plaintexts)
print(cpa_result)

verify_CPA(correct_key, cpa_result)